<a href="https://colab.research.google.com/github/SLCFLAB/hd2025_time_series/blob/main/day3/Day3_4_PriceForecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 금 가격 예측을 위한 시계열 분석 모델 실습

`neuralforecast`의 `cross_validation` 기능을 활용하여 ARIMA, RNN, LSTM, CNN, NHITS, NBEATS 모델의 1일 롤링 예측 성능을 효율적으로 평가하고 비교 분석합니다.

### 1. 라이브러리 설치

In [ ]:
!pip install neuralforecast statsforecast -q

### 2. 라이브러리 임포트

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from neuralforecast import NeuralForecast
from neuralforecast.models import RNN, LSTM, NHITS, NBEATS, TCN
from statsforecast import StatsForecast
from statsforecast.models import AutoARIMA

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error

### 3. 데이터 준비

금 가격 데이터를 불러와 전처리 및 정규화를 수행합니다. `cross_validation` 함수는 내부적으로 데이터를 분할하므로, 전체 데이터를 사용합니다.

In [ ]:
# 데이터 불러오기
url = 'https://raw.githubusercontent.com/SLCFLAB/hd2025_time_series/refs/heads/main/day4/dataset/gold.csv'
df = pd.read_csv(url)

# 데이터 전처리
df = df.rename(columns={'Date': 'ds', 'Close': 'y'})
df['unique_id'] = 'gold'
df = df[['unique_id', 'ds', 'y']]

# 역정규화를 위해 원본 데이터 복사
df_unscaled = df.copy()

# 데이터 정규화
scaler = MinMaxScaler()
df['y'] = scaler.fit_transform(df['y'].values.reshape(-1,1))

# 훈련/테스트 분할 비율 설정 (실제 분할은 cross_validation에서 수행)
train_size = int(len(df) * 0.8)
test_size = len(df) - train_size

### 4. 모델 정의 및 롤링 예측 (Cross Validation 활용)

`cross_validation` 함수를 사용하여 롤링 예측 백테스트를 수행합니다. `horizon=1`로 설정하고 `step_size=1`로 설정하여 매일 모델을 업데이트하며 다음 날을 예측합니다.

In [ ]:
horizon = 5
input_size = 30 # 예측을 위해 30일 전 데이터까지 참고

# 신경망 모델 정의
neural_models = [
    RNN(h=horizon, input_size=input_size, encoder_hidden_size=50, max_steps=100),
    LSTM(h=horizon, input_size=input_size, encoder_hidden_size=50, max_steps=100),
    TCN(h=horizon, input_size=input_size, encoder_hidden_size=50, max_steps=100), ## CNN의 Time Series 특화 버전
    NBEATS(h=horizon, input_size=input_size, max_steps=100),
    NHITS(h=horizon, input_size=input_size, max_steps=100)
]

# NeuralForecast로 롤링 예측 수행
nf = NeuralForecast(models=neural_models, freq=1)
nf_preds_df = nf.cross_validation(df=df, test_size=test_size, step_size=1, n_windows=None)

# ARIMA 모델 정의 및 롤링 예측 수행
sf = StatsForecast(models=[AutoARIMA()], freq=1)
sf_preds_df = sf.cross_validation(df=df, h = horizon, test_size=test_size, step_size=1, n_windows=None)

# 예측 결과 병합
preds_df = pd.merge(nf_preds_df, sf_preds_df, on=['ds', 'unique_id', 'cutoff', 'y'])

In [ ]:
preds_df = preds_df[preds_df['ds']-preds_df['cutoff']==1]

In [ ]:
preds_df

### 5. 결과 시각화 및 평가

`cross_validation`을 통해 얻은 롤링 예측 결과를 시각화하고, MAE를 계산하여 모델별 성능을 비교합니다.

In [ ]:
# 예측 결과 역정규화
for col in preds_df.columns:
    if col not in ['ds', 'unique_id', 'cutoff']:
        preds_df[col] = scaler.inverse_transform(preds_df[col].values.reshape(-1,1))

# 시각화를 위해 실제 값과 예측 값 분리
y_true_df = df_unscaled[train_size:]
y_preds_df = preds_df.set_index('ds')

# 시각화
plt.figure(figsize=(14, 8))
plt.plot(y_preds_df.index, y_preds_df['y'], label='Actual')
for model in ['AutoARIMA', 'RNN', 'LSTM', 'TCN', 'NBEATS', 'NHITS']:
    plt.plot(y_preds_df.index, y_preds_df[model], label=model)

plt.title('Gold Price 1-Day Rolling Forecast (Optimized)')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.show()

# 성능 평가
mae_scores = {}
y_true = preds_df['y'].values

for model in ['AutoARIMA', 'RNN', 'LSTM', 'TCN', 'NBEATS', 'NHITS']:
    y_pred = preds_df[model].values
    mae = mean_absolute_error(y_true, y_pred)
    mae_scores[model] = mae
    print(f'{model} MAE: {mae:.4f}')

# 성능 비교 시각화
plt.figure(figsize=(10, 6))
pd.Series(mae_scores).sort_values().plot(kind='barh')
plt.title('Model MAE Comparison (1-Day Rolling Forecast)')
plt.xlabel('Mean Absolute Error')
plt.show()